# Loading required libraries

In [40]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import json
import matplotlib.pyplot as plt
from shapely import geometry

# Importing Armenia Map Coordinates

In [41]:
with open(r"C:\Users\Karen\Desktop\MappingStaff\armenia.json", errors='ignore') as json_file:
    mapdata = json.load(json_file)

In [43]:
mapdata.keys()

dict_keys(['type', 'arcs', 'transform', 'objects'])

In [30]:
data=pd.read_csv(r"C:\Users\Karen\Desktop\MappingStaff\Coordinates.csv")
data=data[['Cell ID', 'Lat0', 'Lon0','Lat1', 'Lon1','Lat2', 'Lon2','IMSI']]
data.head(2)


,Cell ID,Lat0,Lon0,Lat1,Lon1,Lat2,Lon2,IMSI
0,43011,40.267806,44.617,40.275294,44.626051,40.260317,44.626165,8095
1,43012,40.267806,44.617,40.257444,44.613480,40.270413,44.604741,9412


## Creating shapes for BTS

In [31]:
poly = []
for i in range(len(data)):
    p0 = geometry.Point(data['Lat0'][i],data['Lon0'][i])
    p01 = geometry.Point(data['Lat1'][i],data['Lon1'][i])
    p02 = geometry.Point(data['Lat2'][i],data['Lon2'][i])
    pointList=[p0,p01,p02,p0]
    triang=geometry.Polygon([[p.x, p.y] for p in pointList])
    poly.append(triang)
#append new shapes to the main data
# data['geometry']=poly
data.head(2)

,Cell ID,Lat0,Lon0,Lat1,Lon1,Lat2,Lon2,IMSI
0,43011,40.267806,44.617,40.275294,44.626051,40.260317,44.626165,8095
1,43012,40.267806,44.617,40.257444,44.613480,40.270413,44.604741,9412


In [38]:
geodata=gpd.GeoDataFrame(data, geometry=poly)

In [39]:
import geopandas as gpd
study_area = json.loads("""
 {"type": "FeatureCollection", "features": [{"type": "Feature", "properties": {}, "geometry": {"type": "Polygon", "coordinates": [[[36.394272, -18.626726], [36.394272, -18.558391], [36.489716, -18.558391], [36.489716, -18.626726], [36.394272, -18.626726]]]}}]}
""")
gdf = gpd.GeoDataFrame.from_features(study_area["features"])
print(gdf.head())

                                            geometry
0  POLYGON ((36.394272 -18.626726, 36.394272 -18....
